In [1]:
import pandas as pd
import duckdb

In [2]:
db_location = 'data/nyc_parking_violations.db'

In [13]:
sql_query = "show tables"
with duckdb.connect('data/nyc_parking_violations.db') as con:
    df_ = con.sql(sql_query).df()
    display(df_)

,name
0,bronze_parking_violation_codes
1,bronze_parking_violations
2,first_model
3,gold_ticket_metrics
4,gold_vehicles_metrics
5,parking_violation_2023
6,parking_violation_codes
7,ref_model
8,silver_parking_violations
9,silver_parking_violations_codes


In [39]:
#creating the parking_violationj_codes table again
sql_query_import_1 = '''
create or replace table parking_violation_2023 as 
select * 
from read_csv_auto(
    'data/parking_violations_issued_fiscal_year_2023_sample.csv',
    normalize_names=True)
'''
with duckdb.connect(db_location) as con:
    con.sql(sql_query_import_1)

In [15]:
#querying the created table
sql_query_import_2 = '''SELECT
    violation_code,
    SUM(fee_usd) AS total_revenue_usd
FROM
    silver_parking_violation_codes
GROUP BY
    violation_code
HAVING
    NOT(total_revenue_usd >= 1)'''
with duckdb.connect(db_location) as con:
    display(con.sql(sql_query_import_2))

CatalogException: Catalog Error: Table with name silver_parking_violation_codes does not exist!
Did you mean "silver_parking_violations_codes"?

# Creating prod database

In [ ]:
#creating the parking_violationj_codes table again
db_prod_location = 'data/prod_nyc_parking_violations.db'
sql_query_import_1 = '''
create or replace table parking_violation_2023 as 
select * 
from read_csv_auto(
    'data/dof_parking_violation_codes.csv',
    normalize_names=True)
'''
with duckdb.connect(db_prod_location) as con:
    con.sql(sql_query_import_1)